# Import ERA-5 Climate data and filter by trap locations to reduce size

In [1]:
import numpy as np
from datetime import datetime

In [2]:
import pandas as pd

In [3]:
import xarray as xr
import flox

/Users/katherinedixon/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
faz = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/Arizona_UV_allT.nc", decode_times = True)
fco = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/Colorado_UV_allT.nc", decode_times = True)
fnm = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/NewMexico_UV_allT.nc", decode_times = True)
fpnw = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/PNW_UV_allT.nc", decode_times = True)
fpnw2 = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/PNW_UV_newT.nc", decode_times = True)

In [5]:
fca = xr.open_dataset("/Volumes/My Book/Climate/UV/ERA/California_UV_allT.nc", decode_times = True)

In [ ]:
fmissing1 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates//OR_UV_missing1_ay.nc", decode_times = True)
fmissing2 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates//OR_UV_missing2_ay.nc", decode_times = True)

In [ ]:
fpnw.msdwuvrf

In [6]:
trap_data = pd.read_csv("/Volumes/My Book/Synchrony/_data/ll_info_23.csv")

In [7]:
def downsample_df(dataset):
    
    # faster to do this on the xarray
    dataset['year'] = dataset['time'].dt.strftime('%Y')
    dataset['month'] = dataset['time'].dt.strftime('%B')
    dataset['day'] = dataset['time'].dt.strftime('%d')
    dataset['time'] = dataset['time'].dt.strftime('%r')
    
    df = dataset.to_dataframe()
    df = df.reset_index()
    
    return(df)

In [8]:
dfaz = downsample_df(faz)
dfco = downsample_df(fco)
dfnm = downsample_df(fnm)
dfpnw = downsample_df(fpnw)


In [9]:
dfca = downsample_df(fca)
dfpnw2 = downsample_df(fpnw2)

In [ ]:
dfmissing1 = downsample_df(fmissing1)
dfmissing2 = downsample_df(fmissing2)

In [ ]:
max(dfpnw.longitude)

In [ ]:
dfaz.to_csv("/Volumes/My Book/Climate/UV/downsampled/Arizona_UV_allyearsT.csv")
dfco.to_csv("/Volumes/My Book/Climate/UV/downsampled/Colorado_UV_allyearsT.csv")
dfnm.to_csv("/Volumes/My Book/Climate/UV/downsampled/NewMexico_UV_allyearsT.csv")
dfpnw.to_csv("/Volumes/My Book/Climate/UV/downsampled/PNW_UV_allyearsT.csv")

In [ ]:
dfca.to_csv("/Volumes/My Book/Climate/UV/downsampled/California_UV_allyears_allT.csv")

In [10]:
dfpnw2.to_csv("/Volumes/My Book/Climate/UV/downsampled/PNW_UV_allyears_newT.csv")

In [11]:
df_all = pd.concat([dfaz,dfco,dfnm,dfpnw,dfca,dfpnw2])

In [ ]:
df_all = pd.concat([dfmissing1,dfmissing2])

### Filtering by traps in the state

In [12]:
state_df = trap_data.reset_index(drop=True)

In [13]:
trap_lats = state_df.lat
trap_lons = state_df.lon

print(trap_lats)

0       34.452258
1       34.429790
2       33.861750
3       33.816588
4       33.812968
          ...    
1771    45.897575
1772    45.861841
1773    45.826436
1774    45.893359
1775    45.617810
Name: lat, Length: 1776, dtype: float64


In [14]:
min(trap_lats)

32.427858

In [15]:
grid_lon = list(set(df_all.longitude))
grid_lat = list(set(df_all.latitude))

In [16]:
lat_coord = []

for t in range(0,len(trap_lats)):
    lat_coord.append(grid_lat[np.where(abs(trap_lats[t]- grid_lat) == min(abs(trap_lats[t]- grid_lat)))[0][0]])
    
lon_coord = []

for t in range(0,len(trap_lats)):
    lon_coord.append(grid_lon[np.where(abs(trap_lons[t]- grid_lon) == min(abs(trap_lons[t]- grid_lon)))[0][0]])

In [17]:
print(min(lat_coord),max(lat_coord))
print(min(lon_coord),max(lon_coord))

32.5 51.5
-123.75 -104.75


In [18]:
coord_df = pd.DataFrame({'lat' : trap_lats, 'lon' : trap_lons,
             'lat_coord' : lat_coord, 'lon_coord': lon_coord})

In [ ]:
coord_df

In [19]:
state_df = pd.merge(state_df,coord_df)

In [20]:
state_df["key"] = state_df["lat_coord"].astype(str) + state_df["lon_coord"].astype(str)

In [21]:
state_keys = list(set(state_df.key))

In [22]:
state_keys

['39.5-105.0',
 '50.75-120.5',
 '48.5-120.5',
 '44.0-115.75',
 '48.5-117.75',
 '47.75-118.25',
 '39.25-106.75',
 '44.5-120.0',
 '44.5-117.0',
 '50.25-120.5',
 '42.25-123.5',
 '44.25-117.25',
 '50.5-121.25',
 '48.25-119.25',
 '48.25-120.5',
 '38.5-120.5',
 '48.75-118.25',
 '44.75-119.75',
 '35.25-106.5',
 '44.25-119.25',
 '47.25-116.5',
 '46.0-121.25',
 '45.0-117.25',
 '40.25-121.5',
 '45.25-121.75',
 '38.5-120.0',
 '42.75-121.25',
 '47.25-120.75',
 '47.75-114.0',
 '45.5-121.5',
 '44.75-118.5',
 '44.25-119.5',
 '48.75-119.75',
 '51.25-121.5',
 '48.25-120.75',
 '49.25-119.75',
 '45.0-118.0',
 '34.25-117.0',
 '40.0-120.5',
 '42.0-122.25',
 '49.75-119.5',
 '42.25-123.25',
 '45.75-117.0',
 '43.75-121.75',
 '48.25-120.0',
 '39.25-106.0',
 '49.25-120.0',
 '45.25-116.75',
 '44.75-118.25',
 '48.75-118.75',
 '46.25-117.25',
 '48.5-119.25',
 '45.25-117.0',
 '47.5-121.0',
 '48.25-117.75',
 '48.0-117.25',
 '44.75-115.5',
 '43.0-116.75',
 '44.75-115.75',
 '45.0-117.5',
 '46.0-116.5',
 '44.5-119.75',

In [23]:
df_all["key"] = df_all["latitude"].astype(str) + df_all["longitude"].astype(str)

In [24]:
df_all

,longitude,latitude,time,msdwuvrf,year,month,day,expver,key
0,-112.5,34.5,05:00:00 AM,0.000000,1979,January,01,NaN,34.5-112.5
1,-112.5,34.5,09:00:00 PM,57.258343,1979,January,01,NaN,34.5-112.5
2,-112.5,34.5,05:00:00 AM,0.000000,1979,January,02,NaN,34.5-112.5
3,-112.5,34.5,09:00:00 PM,49.625664,1979,January,02,NaN,34.5-112.5
4,-112.5,34.5,05:00:00 AM,0.000000,1979,January,03,NaN,34.5-112.5
...,...,...,...,...,...,...,...,...,...
10420555,-109.0,42.0,09:00:00 PM,24.538761,2023,November,23,5.0,42.0-109.0
10420556,-109.0,42.0,05:00:00 AM,0.000000,2023,November,24,5.0,42.0-109.0
10420557,-109.0,42.0,09:00:00 PM,33.832100,2023,November,24,5.0,42.0-109.0
10420558,-109.0,42.0,05:00:00 AM,0.000000,2023,November,25,5.0,42.0-109.0


In [25]:
trap_all = df_all[df_all['key'].isin(state_keys)]

In [26]:
trap_all

,longitude,latitude,time,msdwuvrf,year,month,day,expver,key
0,-112.50,34.5,05:00:00 AM,0.000000,1979,January,01,NaN,34.5-112.5
1,-112.50,34.5,09:00:00 PM,57.258343,1979,January,01,NaN,34.5-112.5
2,-112.50,34.5,05:00:00 AM,0.000000,1979,January,02,NaN,34.5-112.5
3,-112.50,34.5,09:00:00 PM,49.625664,1979,January,02,NaN,34.5-112.5
4,-112.50,34.5,05:00:00 AM,0.000000,1979,January,03,NaN,34.5-112.5
...,...,...,...,...,...,...,...,...,...
10196047,-109.25,45.0,09:00:00 PM,17.823833,2023,November,23,5.0,45.0-109.25
10196048,-109.25,45.0,05:00:00 AM,0.000000,2023,November,24,5.0,45.0-109.25
10196049,-109.25,45.0,09:00:00 PM,37.186699,2023,November,24,5.0,45.0-109.25
10196050,-109.25,45.0,05:00:00 AM,0.000000,2023,November,25,5.0,45.0-109.25


In [27]:
trap_all.to_csv("/Volumes/My Book/Climate/UV/downsampled/trap_uv_data_allT.csv")

In [ ]:
trap_all.to_csv("/Volumes/My Book/Climate/UV/downsampled/trap_uv_data_missing.csv")

## Forgot about California when doing this before

In [ ]:
### Filtering by traps in the state

state_df = trap_data.loc[trap_data['State'] == "CA"] 

state_df = state_df.reset_index(drop=True)

trap_lats = state_df.lat
trap_lons = state_df.lon

print(trap_lats)

min(trap_lats)

grid_lon = list(set(dfca.longitude))
grid_lat = list(set(dfca.latitude))

lat_coord = []

for t in range(0,len(trap_lats)):
    lat_coord.append(grid_lat[np.where(abs(trap_lats[t]- grid_lat) == min(abs(trap_lats[t]- grid_lat)))[0][0]])
    
lon_coord = []

for t in range(0,len(trap_lats)):
    lon_coord.append(grid_lon[np.where(abs(trap_lons[t]- grid_lon) == min(abs(trap_lons[t]- grid_lon)))[0][0]])

print(lat_coord)
print(lon_coord)


coord_df = pd.DataFrame({'lat' : trap_lats, 'lon' : trap_lons,
             'lat_coord' : lat_coord, 'lon_coord': lon_coord})

state_df = pd.merge(state_df,coord_df)

state_df["key"] = state_df["lat_coord"].astype(str) + state_df["lon_coord"].astype(str)

state_keys = list(set(state_df.key))

state_df

In [ ]:
dfca["key"] = dfca["latitude"].astype(str) + dfca["longitude"].astype(str)

cali = dfca[dfca['key'].isin(state_keys)]

cali = cali.reset_index(drop=True)

cali

In [ ]:
cali.to_csv("/Volumes/My Book/Climate/UV/downsampled/cali_uv_data.csv")